In [1]:
import pandas as pd
import os
import difflib
import subprocess

In [2]:
APPROACH = "gpt4"

In [3]:
def collect_cocci_files(path: str, approach: str)->list:
    outputs = []
    for root, dirnames, files in os.walk(path):
        for f in files:
            if f.endswith(f".{approach}.cocci"):
                # f = f.split(".")[:-1]
                # f = ".".join(f)
                outputs.append(
                    {
                        "root": root,
                        "cocci_filename": f
                    }
                )
    return pd.DataFrame(outputs)

In [4]:
PATH = "."
df = collect_cocci_files(PATH, APPROACH)

In [5]:
df.cocci_filename.value_counts()

cocci_filename
sp_out.final.gpt4.cocci    14
Name: count, dtype: int64

In [6]:
def collect_c_files(path: str)->list:
    outputs = []
    for root, dirnames, files in os.walk(path):
        for f in files:
            if f.endswith(f".res.c.sanitized.res.c") and "gpt" not in f and "spinfer" not in f and "deepseek" not in f and "claude" not in f:
                f = f.split(".")[0]
                outputs.append(
                    {
                        "root": root,
                        "c_filename": f
                    }
                )
    return pd.DataFrame(outputs)

In [7]:
df_c = collect_c_files(PATH)
df_c.root.value_counts()

root
./tcf_block_get-61        27
./snd_soc                 20
./dasd_smalloc            18
./kees_timer1             16
./EXP0-7                  15
./random_ether_addr-84    12
./perf_evlist__mmap-69    11
./sock_poll_wait-84       10
./free_bootmem-77         10
./tcaction                 8
./early_memunmap           8
./ttm_bo_init-60           6
./dma_pool_alloc-52        6
./uartlite                 3
Name: count, dtype: int64

In [8]:
df_c.c_filename.to_list()

['1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrfs_qgroup_free_meta_all_pertrans',
 '1533533124_2018-08-06_4fd786e6c3d6_super_btrfs_statfs',
 '1533533124_2018-08-06_4fd786e6c3d6_ctree_add_root_to_dirty_list',
 '1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrfs_qgroup_reserve_data',
 '1533533124_2018-08-06_4fd786e6c3d6_relocation_build_backref_tree',
 '1533533124_2018-08-06_4fd786e6c3d6_qgroup_qgroup_free_reserved_data',
 '1533533124_2018-08-06_4fd786e6c3d6_send_send_subvol_begin',
 '1533533124_2018-08-06_4fd786e6c3d6_ref-verify_btrfs_ref_tree_mod',
 '1533533124_2018-08-06_4fd786e6c3d6_backref_iterate_inode_refs',
 '1533533124_2018-08-06_4fd786e6c3d6_qgroup___btrfs_qgroup_reserve_meta',
 '1533533124_2018-08-06_4fd786e6c3d6_ioctl_btrfs_ioctl_default_subvol',
 '1533533124_2018-08-06_4fd786e6c3d6_qgroup___btrfs_qgroup_free_meta',
 '1533533124_2018-08-06_4fd786e6c3d6_extent-tree_btrfs_drop_snapshot',
 '1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrfs_qgroup_convert_reserved_meta',
 '1533533124_

In [9]:
for i, row in df_c.sample(n=5, random_state=123).iterrows():
    print(row["root"], row["c_filename"])

./uartlite 1360605874_2013-02-11_6d53c3b71d32_uartlite_ulite_tx_empty
./snd_soc 1524629994_2018-04-25_e4b31b816c47_mt2701-afe-pcm_mt2701_simple_fe_startup
./free_bootmem-77 1540937361_2018-10-30_2013288f7238_setup_64_pcpu_fc_free
./random_ether_addr-84 1529689860_2018-06-22_6c1f0a1ffb7c_hip04_eth_hip04_mac_probe
./tcaction 1534706529_2018-08-19_244cd96adb5f_bnxt_tc_bnxt_tc_parse_actions


In [10]:
for i, row in df.sample(n=5, random_state=123).iterrows():
    print(row["root"], row["cocci_filename"])

./perf_evlist__mmap-69 sp_out.final.gpt4.cocci
./dma_pool_alloc-52 sp_out.final.gpt4.cocci
./snd_soc sp_out.final.gpt4.cocci
./EXP0-7 sp_out.final.gpt4.cocci
./ttm_bo_init-60 sp_out.final.gpt4.cocci


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   root            14 non-null     object
 1   cocci_filename  14 non-null     object
dtypes: object(2)
memory usage: 356.0+ bytes


In [12]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   root        170 non-null    object
 1   c_filename  170 non-null    object
dtypes: object(2)
memory usage: 2.8+ KB


In [13]:
df = pd.merge(df, df_c, on='root', how='inner')
df.drop_duplicates(inplace=True)
df.shape

(170, 3)

In [14]:
df["pred"] = df["cocci_filename"].apply(lambda x: x.split(".")[0])
df["pred"] = df["pred"].apply(lambda x: x.split("_")[-1])
df.head(20)

,root,cocci_filename,c_filename,pred
0,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrf...,out
1,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_super_btrfs...,out
2,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_ctree_add_r...,out
3,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrf...,out
4,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_relocation_...,out
5,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup_qgro...,out
6,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_send_send_s...,out
7,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_ref-verify_...,out
8,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_backref_ite...,out
9,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup___bt...,out


# check syntax correctness

In [15]:
def check_syntax_correctness(filepath: str)->bool:
    '''
    Validates a Coccinelle semantic patch by parsing it, writes debug information, and returns a boolean indicating the validity.
    '''
    command = f'spatch --parse-cocci {filepath}'.split()

    try:
        result = subprocess.run(command, check=True, capture_output=True)
        return 1
    except Exception as e:
        print(e)
        return 0

In [16]:
df["is_cocci_valid"] = df.apply(lambda x: check_syntax_correctness(os.path.join(x.root, x.cocci_filename)), axis=1)

Command '['spatch', '--parse-cocci', './tcf_block_get-61/sp_out.final.gpt4.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/sp_out.final.gpt4.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/sp_out.final.gpt4.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/sp_out.final.gpt4.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/sp_out.final.gpt4.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/sp_out.final.gpt4.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/sp_out.final.gpt4.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_block_get-61/sp_out.final.gpt4.cocci']' returned non-zero exit status 255.
Command '['spatch', '--parse-cocci', './tcf_bloc

# check how many cocci is successfully applied

In [17]:
df["is_applied"] = df.apply(lambda x:os.path.exists(x.root + "/" + x.c_filename+f".{x.pred}.{APPROACH}.res.c.sanitized.res.c"), axis=1)
df.is_applied.value_counts()

is_applied
False    89
True     81
Name: count, dtype: int64

# compute precision and recall

In [18]:
def open_and_read_file_content(filepath: str)->str:
    """Open the input filepath, then read the content"""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
    except IOError as e:
        print(f"An error occurred while reading the file: {e}")
    return ""

def remove_unmodified_lines(input_diff: list, is_merge: bool = True)->str:
    additions = []
    removals = []
    for line in input_diff:
        if line.startswith("+") and "+++" not in line:
            additions.append(line)
        elif line.startswith("-") and "---" not in line:
            removals.append(line)
    output_str = removals + additions
    if is_merge:
        return "\n".join(output_str)
    else:
        return removals + additions

def get_diff(file_a: str, file_b: str, n_context: int)->str:
    """Get the diff between file_a and file_b using difflib with n_context length"""
    file_a = open_and_read_file_content(file_a)
    if file_a == "" or "parse error :" in file_a or "init_defs_builtins : " in file_a:
        return ""
    file_b = open_and_read_file_content(file_b)
    if file_b == "" or "parse error :" in file_b or "init_defs_builtins : " in file_b:
        return ""

    lines_a = file_a.splitlines()
    lines_b = file_b.splitlines()        
    # Generate the unified diff
    diff = difflib.unified_diff(
        lines_a,
        lines_b,
        fromfile="initial",
        tofile="final",
        lineterm='',
        n=n_context
    )
    # diff = [x for x in diff if not x.startswith("+++") and not x.startswith("---") and not x.startswith("@@")]
    if n_context == 0:
        diff = remove_unmodified_lines(input_diff=diff, is_merge=False)
    # Convert the diff generator to a single string
    diff_text = '\n'.join(diff)
    return diff_text

In [19]:
df["diff_truth"] = df.apply(lambda x:get_diff(
    file_a = os.path.join(x.root, x.c_filename + ".c.sanitized.c"),
    file_b = os.path.join(x.root, x.c_filename + ".res.c.sanitized.res.c"),
    n_context = 0
), axis=1)

In [20]:
df

,root,cocci_filename,c_filename,pred,is_cocci_valid,is_applied,diff_truth
0,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrf...,out,1,True,"- if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &fs_i..."
1,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_super_btrfs...,out,1,True,- buf->f_fsid.val[0] ^= BTRFS_I(d_inode(dentr...
2,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_ctree_add_r...,out,1,True,- if (root->objectid == BTRFS_EXTENT_TREE...
3,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrf...,out,1,True,"- if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &root..."
4,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_relocation_...,out,1,True,"- btrfs_err(root->fs_info, ""couldn't ..."
...,...,...,...,...,...,...,...
165,./dasd_smalloc,sp_out.final.gpt4.cocci,1528132059_2018-06-04_c5205f2ff2be_dasd_eckd_d...,out,0,False,"- cqr = dasd_smalloc_request(DASD_ECKD_MAGIC,..."
166,./dasd_smalloc,sp_out.final.gpt4.cocci,1528132059_2018-06-04_c5205f2ff2be_dasd_dasd_g...,out,0,False,"- cqr = dasd_smalloc_request(magic, 1, rdc_bu..."
167,./dasd_smalloc,sp_out.final.gpt4.cocci,1528132059_2018-06-04_c5205f2ff2be_dasd_eckd_d...,out,0,False,"- cqr = dasd_smalloc_request(DASD_ECKD_MAGIC,..."
168,./dasd_smalloc,sp_out.final.gpt4.cocci,1528132059_2018-06-04_c5205f2ff2be_dasd_eckd_d...,out,0,False,"- cqr = dasd_smalloc_request(DASD_ECKD_MAGIC,..."


In [21]:
df["diff_pred"] = df.apply(lambda x:get_diff(
    file_a = os.path.join(x.root, x.c_filename + ".c.sanitized.c"),
    file_b = os.path.join(x.root, x.c_filename + f".{x.pred}.{APPROACH}.res.c.sanitized.res.c"),
    n_context = 0
), axis=1)

Error: The file './tcf_block_get-61/1507896057_2017-10-13_69d78ef25c7b_sch_qfq_qfq_init_qdisc.out.gpt4.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1513791319_2017-12-20_8d1a77f974ca_sch_dsmark_dsmark_init.out.gpt4.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1513791319_2017-12-20_8d1a77f974ca_sch_sfb_sfb_init.out.gpt4.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1507896057_2017-10-13_69d78ef25c7b_sch_cbq_cbq_change_class.out.gpt4.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1507896057_2017-10-13_69d78ef25c7b_sch_atm_atm_tc_init.out.gpt4.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1513791319_2017-12-20_8d1a77f974ca_sch_atm_atm_tc_init.out.gpt4.res.c.sanitized.res.c' was not found.
Error: The file './tcf_block_get-61/1507896057_2017-10-13_69d78ef25c7b_sch_hfsc_hfsc_init_qdisc.out.gpt4.res.c.sanitized.res.c' was not found.
Error: The file './tc

In [22]:
df["diff_5"] = df.apply(lambda x:get_diff(
    file_a = os.path.join(x.root, x.c_filename + ".c.sanitized.c"),
    file_b = os.path.join(x.root, x.c_filename + f".res.c.sanitized.res.c"),
    n_context = 5
), axis=1)

In [23]:
df

,root,cocci_filename,c_filename,pred,is_cocci_valid,is_applied,diff_truth,diff_pred,diff_5
0,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrf...,out,1,True,"- if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &fs_i...","- if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &fs_i...","--- initial\n+++ final\n@@ -1,10 +1,10 @@\n vo..."
1,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_super_btrfs...,out,1,True,- buf->f_fsid.val[0] ^= BTRFS_I(d_inode(dentr...,- buf->f_fsid.val[0] ^= BTRFS_I(d_inode(dentr...,"--- initial\n+++ final\n@@ -65,9 +65,9 @@\n ..."
2,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_ctree_add_r...,out,1,True,- if (root->objectid == BTRFS_EXTENT_TREE...,- if (root->objectid == BTRFS_EXTENT_TREE...,"--- initial\n+++ final\n@@ -5,11 +5,11 @@\n ..."
3,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrf...,out,1,True,"- if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &root...","- if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &root...","--- initial\n+++ final\n@@ -5,11 +5,11 @@\n ..."
4,./EXP0-7,sp_out.final.gpt4.cocci,1533533124_2018-08-06_4fd786e6c3d6_relocation_...,out,1,True,"- btrfs_err(root->fs_info, ""couldn't ...","- btrfs_err(root->fs_info, ""couldn't ...","--- initial\n+++ final\n@@ -212,11 +212,11 @@\..."
...,...,...,...,...,...,...,...,...,...
165,./dasd_smalloc,sp_out.final.gpt4.cocci,1528132059_2018-06-04_c5205f2ff2be_dasd_eckd_d...,out,0,False,"- cqr = dasd_smalloc_request(DASD_ECKD_MAGIC,...",,"--- initial\n+++ final\n@@ -1,11 +1,11 @@\n st..."
166,./dasd_smalloc,sp_out.final.gpt4.cocci,1528132059_2018-06-04_c5205f2ff2be_dasd_dasd_g...,out,0,False,"- cqr = dasd_smalloc_request(magic, 1, rdc_bu...",,"--- initial\n+++ final\n@@ -1,11 +1,11 @@\n st..."
167,./dasd_smalloc,sp_out.final.gpt4.cocci,1528132059_2018-06-04_c5205f2ff2be_dasd_eckd_d...,out,0,False,"- cqr = dasd_smalloc_request(DASD_ECKD_MAGIC,...",,"--- initial\n+++ final\n@@ -15,11 +15,11 @@\n ..."
168,./dasd_smalloc,sp_out.final.gpt4.cocci,1528132059_2018-06-04_c5205f2ff2be_dasd_eckd_d...,out,0,False,"- cqr = dasd_smalloc_request(DASD_ECKD_MAGIC,...",,"--- initial\n+++ final\n@@ -3,11 +3,11 @@\n ..."


In [24]:
def compute_precision(truths: list, results: list)->int:
    '''
    Computes the precision of the results compared to the truths, rounding the result to three decimal places.
    '''
    truths_cp = truths.copy()
    is_match = []
    for line in results:
        if line in truths_cp:
            truths_cp.remove(line)
            is_match.append(1)
        else:
            is_match.append(0)
    precision = sum(is_match)/len(is_match) if len(is_match) != 0 else 0
    return round(precision, 3)

In [25]:
df["precision"] = df.apply(lambda x: compute_precision(
    x.diff_truth.splitlines(),
    x.diff_pred.splitlines()
), axis=1)

In [26]:
def compute_recall(truths: list, results: list)->int:
    '''
    Computes the recall of the results compared to the truths, rounding the result to three decimal places.
    '''
    results_cp = results.copy()
    is_match = []
    for line in truths:
        if line in results_cp:
            results_cp.remove(line)
            is_match.append(1)
        else:
            is_match.append(0)
    recall = sum(is_match)/len(is_match) if len(is_match) != 0 else 0
    return round(recall, 3)

In [27]:
df["recall"] = df.apply(lambda x: compute_recall(
    x.diff_truth.splitlines(),
    x.diff_pred.splitlines()
), axis=1)

In [28]:
df["cocci_fullpath"] = df.apply(lambda x: os.path.join(x.root, x.cocci_filename), axis=1)

# check final example coverage

- Here, we only care about the precision and recall
    - If tie, then choose the highest one


- the selection of the best patch in the previous step guarantee that the patch will also yield similar result as the choosing the highest precision and recall 
    - note that we also use precision and recall as one of the criterion when selecting the best patch
    - but, the selection process is more refined (i check the overfitting issue)

In [29]:
df.c_filename.nunique()

170

In [30]:
df['rank_score'] = (df['precision'] + df['recall'] + df["is_cocci_valid"]) / 3
best_preds_per_cluster = df.loc[df.groupby(['c_filename'])['rank_score'].idxmax()]
best_clusters = best_preds_per_cluster.loc[
    best_preds_per_cluster.groupby('c_filename')['rank_score'].idxmax()
]
best_clusters = best_clusters.drop(columns=['rank_score'])

In [31]:
best_clusters.shape

(170, 12)

In [32]:
best_clusters

,root,cocci_filename,c_filename,pred,is_cocci_valid,is_applied,diff_truth,diff_pred,diff_5,precision,recall,cocci_fullpath
92,./uartlite,sp_out.final.gpt4.cocci,1360605874_2013-02-11_6d53c3b71d32_uartlite_ul...,out,1,True,- val = ioread32be(port->membase + ULITE_...,- val = ioread32be(port->membase + ULITE_...,"--- initial\n+++ final\n@@ -3,11 +3,11 @@\n ...",1.0,1.0,./uartlite/sp_out.final.gpt4.cocci
90,./uartlite,sp_out.final.gpt4.cocci,1360605874_2013-02-11_6d53c3b71d32_uartlite_ul...,out,1,True,-static void ulite_start_tx(struct uart_port *...,-static void ulite_start_tx(struct uart_port *...,--- initial\n+++ final\n@@ -1 +1 @@\n-static v...,1.0,1.0,./uartlite/sp_out.final.gpt4.cocci
91,./uartlite,sp_out.final.gpt4.cocci,1360605874_2013-02-11_6d53c3b71d32_uartlite_ul...,out,1,True,- ret = ioread32be(port->membase + ULITE_STAT...,- ret = ioread32be(port->membase + ULITE_STAT...,"--- initial\n+++ final\n@@ -1,9 +1,9 @@\n stat...",1.0,1.0,./uartlite/sp_out.final.gpt4.cocci
108,./early_memunmap,sp_out.final.gpt4.cocci,1402317686_2014-06-09_98a716b66cab_quirks_efi_...,out,1,True,"- early_iounmap(tablep, nr_tables * sz);\n- ...","- early_iounmap(tablep, nr_tables * sz);\n- ...","--- initial\n+++ final\n@@ -29,11 +29,11 @@\n ...",1.0,1.0,./early_memunmap/sp_out.final.gpt4.cocci
105,./early_memunmap,sp_out.final.gpt4.cocci,1402317686_2014-06-09_98a716b_quirks_efi_reuse...,out,1,True,"- early_iounmap(tablep, nr_tables * sz);\n- ...","- early_iounmap(tablep, nr_tables * sz);\n- ...","--- initial\n+++ final\n@@ -29,11 +29,11 @@\n ...",1.0,1.0,./early_memunmap/sp_out.final.gpt4.cocci
...,...,...,...,...,...,...,...,...,...,...,...,...
136,./free_bootmem-77,sp_out.final.gpt4.cocci,1540937361_2018-10-30_2013288f7238_smp_64_pcpu...,out,1,True,-static void __init pcpu_free_bootmem(void *pt...,-static void __init pcpu_free_bootmem(void *pt...,--- initial\n+++ final\n@@ -1 +1 @@\n-static v...,1.0,1.0,./free_bootmem-77/sp_out.final.gpt4.cocci
134,./free_bootmem-77,sp_out.final.gpt4.cocci,1540937361_2018-10-30_2013288f7238_smu_smu_init,out,1,True,"- free_bootmem(__pa(smu), sizeof(struct smu_d...","- free_bootmem(__pa(smu), sizeof(struct smu_d...","--- initial\n+++ final\n@@ -78,11 +78,11 @@\n ...",1.0,1.0,./free_bootmem-77/sp_out.final.gpt4.cocci
135,./free_bootmem-77,sp_out.final.gpt4.cocci,1540937361_2018-10-30_2013288f7238_swiotlb-xen...,out,1,True,"- free_bootmem(__pa(xen_io_tlb_start), ...","- free_bootmem(__pa(xen_io_tlb_start), ...","--- initial\n+++ final\n@@ -38,11 +38,11 @@\n ...",1.0,1.0,./free_bootmem-77/sp_out.final.gpt4.cocci
142,./free_bootmem-77,sp_out.final.gpt4.cocci,1540937361_2018-10-30_2013288f7238_xhci-dbc_ea...,out,1,True,"- free_bootmem(xdbc.table_dma, PAGE_SIZ...","- free_bootmem(xdbc.table_dma, PAGE_SIZ...","--- initial\n+++ final\n@@ -11,13 +11,13 @@\n ...",1.0,1.0,./free_bootmem-77/sp_out.final.gpt4.cocci


In [33]:
round(best_clusters.precision.mean(), 2), round(best_clusters.recall.mean(), 2), round(best_clusters.drop_duplicates(subset=["root"]).is_cocci_valid.mean(), 2)

(np.float64(0.35), np.float64(0.35), np.float64(0.57))

In [34]:
mean_df = best_clusters.groupby(["root"])[["is_cocci_valid", "precision", "recall"]].mean().round(2)
mean_df

,is_cocci_valid,precision,recall
root,,,
./EXP0-7,1.0,0.97,0.97
./dasd_smalloc,0.0,0.00,0.00
./dma_pool_alloc-52,0.0,0.00,0.00
./early_memunmap,1.0,1.00,1.00
./free_bootmem-77,1.0,1.00,1.00
./kees_timer1,1.0,0.12,0.12
./perf_evlist__mmap-69,1.0,0.91,0.91
./random_ether_addr-84,1.0,1.00,1.00
./snd_soc,0.0,0.00,0.00


In [35]:
# Step 1: Group the data by 'root' and 'cocci_filename'
grouped = best_clusters.groupby(['root'])

# Step 2: Compute total samples per group
group_counts = grouped.size().reset_index(name='num_samples')
temp_df = best_clusters.merge(group_counts, on=['root'])

total_samples = temp_df['num_samples'].sum()
weighted_precision = (temp_df['precision'] * temp_df['num_samples']).sum() / total_samples
weighted_recall = (temp_df['recall'] * temp_df['num_samples']).sum() / total_samples


weighted_precision, weighted_recall

(np.float64(0.2593941717791411), np.float64(0.2593941717791411))

In [36]:
mean_df.reset_index(inplace=True)
mean_df["root"] = mean_df["root"].apply(lambda x: x.split("/")[-1])
mean_df.to_csv("result_perdir_gpt.csv", index=False)

In [37]:
df["cocci"] = df.cocci_fullpath.apply(lambda x: open_and_read_file_content(x))

In [38]:
df.to_csv(f"computed_{APPROACH}.csv", index=False)